__ProPythia__

This file is a simulation for the antioxidant dataset;

I intend to run propythia here so that in the future I can make comparisons with the results obtained with omnia.

i will do this simulation with the antioxidant data, where the data in unbalanced

# imports

In [0]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from scipy.stats import loguniform
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef,f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

from propythia.protein.descriptors import ProteinDescritors
from propythia.protein.encoding import Encoding
from propythia.ml.shallow_ml import ShallowML
from propythia.protein.sequence import ReadSequence

from propythia.ml.deep_ml import DeepML

from propythia.feature_selection import FeatureSelection

from propythia.preprocess import Preprocess

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


# read the data is the first step !! do not forget

In [0]:
read_seqs = ReadSequence()
x_train_nlf = read_seqs.par_preprocessing(dataset= x_train_nlf, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
x_test_nlf = read_seqs.par_preprocessing(dataset= x_test_nlf, col = 'sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')


# Encoding

First create a function that allows to padding and truncate the sequence , i choose the value X like the character to be add in case of padding;


next aplicate the encoding nlf classe from propythia

__Note__: 600 is a high value for our dataset, but is the value more mentioned in literature

In [0]:
def pad_and_truncate_sequences(df, seq_col, max_length, padding_value='X'):
    """
    Pad and truncate the protein sequences in a DataFrame to a specific length.

    :param df: DataFrame containing the protein sequences.
    :param seq_col: Name of the column in df that contains the protein sequences.
    :param max_length: The maximum length for all sequences.
    :param padding_value: The value to use for padding the sequences.
    :return: DataFrame with the padded and truncated sequences.
    """
    def pad_and_truncate(seq):
        # Truncate the sequence if it's too long
        if len(seq) > max_length:
            seq = seq[:max_length]
        # Pad the sequence if it's not long enough
        elif len(seq) < max_length:
            seq += padding_value * (max_length - len(seq))
        return seq

    df['padded_and_truncated_sequence'] = df[seq_col].apply(pad_and_truncate)
    return df

# Use the function
x_train_nlf_encode = pad_and_truncate_sequences(x_train_nlf, 'sequence', 600)
x_test_nlf_encode = pad_and_truncate_sequences(x_test_nlf, 'sequence', 600)


In [0]:
enconde_train_df = Encoding(dataset= x_train_nlf_encode ,  col= 'padded_and_truncated_sequence')
enconde_test_df=Encoding(dataset= x_test_nlf_encode ,  col= 'padded_and_truncated_sequence')

In [0]:
#train
nlf_train=enconde_train_df.get_nlf()
nlf_test=enconde_test_df.get_nlf()
expanded_train_arrays =  nlf_train['nlf'].apply(lambda x: np.array(x))
#test
x_train_nlf_ = np.array(expanded_train_arrays.tolist())
expanded_test_arrays =  nlf_test['nlf'].apply(lambda x: np.array(x))
x_test_nlf_ = np.array(expanded_test_arrays.tolist())

In [0]:
x_test_nlf_.shape

# split antioxidant_dataset X and Y 

the goal is to split the datatese in train , test and do the normalization for the x descriptors

We decide to use the same dataset for test and train for all the experiments

In [0]:
#X_train_nlf, X_test_nlf, y_train_nlf, y_test_nlf = train_test_split(X_nlf,y_nlf, test_size=0.2,stratify=y_nlf, random_state=42)

# standard scaler article does not refer scaling and do not validate in x_test, however, we do it anyway
#scaler = StandardScaler().fit(X_train_descriptors)
#X_train_descriptors = scaler.transform(X_train_descriptors)
#X_test_descriptors = scaler.transform(X_test_descriptors)

# Deep Learning

The aim is to recreate the deepLearning models implemented in omnia as well as the possibility of optimising hyperparameters by grid search (using a param_grid).

# RNN

In [0]:
#RNN and propythia
def create_rnn_model(rnn_type='LSTM', bidirectional=False, num_rnn_layers=1, hidden_dim=64, num_dense_layers=1, neurons_dense=32, output_dim=1, drop=0.3, activation='relu', last_layers_activations='sigmoid'):
    model = tf.keras.models.Sequential()

    # RNN layers
    for i in range(num_rnn_layers):
        current_hidden_dim =hidden_dim // (2**i)
        if rnn_type == 'LSTM':
            rnn_layer = tf.keras.layers.LSTM(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        elif rnn_type == 'GRU':
            rnn_layer = tf.keras.layers.GRU(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        elif rnn_type == 'SimpleRNN':
            rnn_layer = tf.keras.layers.SimpleRNN(current_hidden_dim, return_sequences=(i != num_rnn_layers - 1), activation=activation)
        else:
            raise ValueError("Invalid RNN type. Supported types are 'LSTM', 'GRU', and 'SimpleRNN'.")

        if bidirectional:
            rnn_layer = tf.keras.layers.Bidirectional(rnn_layer)

        model.add(rnn_layer)
        model.add(tf.keras.layers.Dropout(drop))

    # Dense layers
    for i in range(num_dense_layers):
        current_neurons_dense = neurons_dense if i == 0 else neurons_dense // (2**i)
        model.add(tf.keras.layers.Dense(current_neurons_dense, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))

    model.add(tf.keras.layers.Dense(output_dim, activation=last_layers_activations))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dl=DeepML(x_train_nlf, y_train_nlf, x_test_nlf, y_test_nlf, number_classes=2, problem_type='binary',
          x_dval=None, y_dval=None, epochs=10, batch_size=32,
          path='', report_name=None, verbose=1,
         early_stopping_patience=2, reduce_lr_patience=10, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                 )
model = KerasClassifier(build_fn=create_rnn_model)


In [0]:
%%capture captured

def generate_param_grid(num_rnn_layers:list, num_dense_layers:list):
    param_grid = {
        'rnn_type': ['LSTM', 'GRU', 'SimpleRNN'],
        'bidirectional': [True, False],
        'num_rnn_layers': num_rnn_layers,
        'hidden_dim': [64,128],
        'num_dense_layers': num_dense_layers,
        'neurons_dense': [64,128],
        'output_dim': [1],
        'drop': [0.1, 0.3],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }
    return param_grid

# Define os parâmetros do grid
num_rnn_layers = [2,3]
num_dense_layers = [2,3]
param_grid = generate_param_grid(num_rnn_layers, num_dense_layers)

# Otimização do modelo
best_classifier_rnn = dl.get_opt_params(param_grid, model, scoring=make_scorer(f1_score), optType='randomizedSearch', cv=5, n_iter_search=50)

# Avaliação do modelo
scores, report, cm, cm2 = dl.score_testset_classification(best_classifier_rnn)

# Predição dos valores para o teste
y_pred = best_classifier_rnn.predict(dl.x_test)

# Matriz de confusão
conf_mat = confusion_matrix(dl.y_test, y_pred)

# Guarda o output capturado num ficheiro
with open('output_data_nlf_rnn.log', 'w') as f:
    f.write(str(captured))

# Guarda os scores num ficheiro .txt
with open('scores_data_nlf_rnn.txt', 'w') as f:
    f.write(report)  # O relatório normalmente já contém f1-score, precisão, recall

# Guarda a confusion matrix num ficheiro .txt ou .csv
np.savetxt('confusion_matrix_data_nlf_rnn.log.csv', conf_mat, delimiter=",")

# Visualiza a matriz de confusão
dl.conf_matrix_seaborn_table(conf_matrix=conf_mat)
